In [9]:
import numpy as np
import pandas as pd
import gc
import os
import sys
sys.path.append("../src")
from config import read_config, update_config
from metric import cal_mae_metric
from dataset import read_data
from FE import add_features_choice
from util import smart_avg,reduce_mem_usage
from sklearn.preprocessing import RobustScaler,MinMaxScaler

In [2]:
config = read_config("LSTM5_CLS_DO02_CH01_IPOnly_SiLU_ADAM_PL")
config.gpu = [0]
config.debug = True
config = update_config(config)

Using device: cuda
Number of device: 1
Model Output Folder: /home/vincent/Kaggle/GBVPP/output/LSTM5_CLS_DO02_CH01_IPOnly_SiLU_ADAM_PL/


In [3]:
train, test = read_data(config)
train.drop(columns="pressure", inplace=True)
all_data = pd.concat([train, test],axis=0).reset_index(drop=True)
all_data.sort_values(["breath_id","id"], inplace=True)
RC_data = all_data.groupby("breath_id")[["R","C"]].last()
if not os.path.exists("../output/clustering/"):
    os.makedirs("../output/clustering/")
RC_data.to_csv("../output/clustering/RC_data.csv")
del train, test
gc.collect()

0

In [4]:
all_data.shape

(204800, 8)

In [5]:
all_data = add_features_choice(all_data.copy(), config)
all_data, NAlist = reduce_mem_usage(all_data)

--- Generate u_out features ---
--- Generate time features ---
--- Generate u_in features ---
--- Generate cross sectional features ---
-- generate physics features --
--- Generate R C features ---
Drop Low Importance Columns: ['step', 'cross_time', 'expand_skew', 'expand_kurt']
Memory usage of properties dataframe is : 69.53125  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is: 48.8 MB
This is 7022.5% of the initial size


In [6]:
all_data.head()

,id,breath_id,time_step,u_in,u_out,fold,u_out_diff,u_out_diff_back1,u_out_diff_back2,time_delta,...,C_50,R_C_20_10,R_C_20_20,R_C_20_50,R_C_50_10,R_C_50_20,R_C_50_50,R_C_5_10,R_C_5_20,R_C_5_50
102400,1,0,0.000000,0.000000,0,0,0,0,0,0.033099,...,0,0,0,0,0,0,0,0,1,0
102401,2,0,0.031904,7.515046,0,0,0,0,0,0.031904,...,0,0,0,0,0,0,0,0,1,0
102402,3,0,0.063827,14.651675,0,0,0,0,0,0.031924,...,0,0,0,0,0,0,0,0,1,0
102403,4,0,0.095751,21.230610,0,0,0,0,0,0.031924,...,0,0,0,0,0,0,0,0,1,0
102404,5,0,0.127644,26.320955,0,0,0,0,0,0.031893,...,0,0,0,0,0,0,0,0,1,0


# Prepare Data

In [7]:
feature_cols = [col for col in all_data.columns if col not in ["id", "breath_id", "fold", "pressure"]]

In [10]:
no_transform_cols = ['u_out', 'u_out_diff', "u_out_diff_back1", "u_out_diff_back2",
                     'R_20', 'R_5', 'R_50', 'C_10', 'C_20', 'C_50',
                     'R_C_20_10', 'R_C_20_20', 'R_C_20_50', 'R_C_50_10', 'R_C_50_20',
                     'R_C_50_50', 'R_C_5_10', 'R_C_5_20', 'R_C_5_50']
transform_cols = [col for col in feature_cols if col not in no_transform_cols]
scaler = RobustScaler(quantile_range=(5, 95), unit_variance=True)
scaler = MinMaxScaler()
all_data[transform_cols] = scaler.fit_transform(all_data[transform_cols])

In [11]:
all_data[feature_cols].max(axis=0)

time_step           1.0
u_in                1.0
u_out               1.0
u_out_diff          1.0
u_out_diff_back1    1.0
                   ... 
R_C_50_20           1.0
R_C_50_50           1.0
R_C_5_10            1.0
R_C_5_20            1.0
R_C_5_50            1.0
Length: 73, dtype: float32

In [12]:
all_data[feature_cols].min(axis=0)

time_step           0.0
u_in                0.0
u_out               0.0
u_out_diff          0.0
u_out_diff_back1    0.0
                   ... 
R_C_50_20           0.0
R_C_50_50           0.0
R_C_5_10            0.0
R_C_5_20            0.0
R_C_5_50            0.0
Length: 73, dtype: float32

In [13]:
all_data = all_data.merge(RC_data,on="breath_id")
all_data

,id,breath_id,time_step,u_in,u_out,fold,u_out_diff,u_out_diff_back1,u_out_diff_back2,time_delta,...,R_C_20_20,R_C_20_50,R_C_50_10,R_C_50_20,R_C_50_50,R_C_5_10,R_C_5_20,R_C_5_50,R,C
0,1,0,0.000000,0.000000,0,0,0,0,0,0.143335,...,0,0,0,0,0,0,1,0,5,20
1,2,0,0.011649,0.075150,0,0,0,0,0,0.038074,...,0,0,0,0,0,0,1,0,5,20
2,3,0,0.023305,0.146517,0,0,0,0,0,0.039817,...,0,0,0,0,0,0,1,0,5,20
3,4,0,0.034962,0.212306,0,0,0,0,0,0.039838,...,0,0,0,0,0,0,1,0,5,20
4,5,0,0.046607,0.263210,0,0,0,0,0,0.037087,...,0,0,0,0,0,0,1,0,5,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204795,102396,3291,0.944138,0.049805,1,0,0,0,0,0.255954,...,0,0,0,0,1,0,0,0,50,50
204796,102397,3291,0.956593,0.049836,1,0,0,0,0,0.232370,...,0,0,0,0,1,0,0,0,50,50
204797,102398,3291,0.969373,0.049862,1,0,0,0,0,0.310912,...,0,0,0,0,1,0,0,0,50,50
204798,102399,3291,0.981915,0.049884,1,0,0,0,0,0.253434,...,0,0,0,0,1,0,0,0,50,50


In [ ]:
all_data.to_csv("../output/clustering/all_data_fe.csv")